# Architecture
> Bidirectional LSTMs focus on the problem of getting the most out of the input sequence by
stepping through input time steps in both the forward and backward directions


# Implementation
- The LSTM layer in Keras allow you to specify the directionality of the input sequence. This can
be done by setting the go backwards argument to True (defaults to False).
```python
model = Sequential()
model.add(LSTM(..., input_shape=(...), go_backwards=True))

- Specifically, Bidirectional LSTMs are supported in Keras via the Bidirectional layer wrapper that essentially merges the output from two parallel LSTMs, one with input processed forward and one with output
processed backwards. This wrapper takes a recurrent layer (e.g. the first hidden LSTM layer) as
an argument.
```
model = Sequential()
model.add(Bidirectional(LSTM(...), input_shape=(...)))
```

- The Bidirectional wrapper layer also allows you to specify the merge mode; that is how
the forward and backward outputs should be combined before being passed on to the next layer.
The options are:
>- ‘sum’: The outputs are added together.  
>- ‘mul’: The outputs are multiplied together.  
>- ‘concat’: The outputs are concatenated together (the default), providing double the
number of outputs to the next layer.  
>- ‘ave’: The average of the outputs is taken.







# Cumulative Sum Prediction Problem


## Cumulative Sum
 - The problem is defined as a sequence of random values between 0 and 1. This sequence is taken
as input for the problem with each number provided once per time step. A binary label (0 or 1)
is associated with each input. The output values are all 0. Once the cumulative sum of the
input values in the sequence exceeds a threshold, then the output value flips from 0 to 1.


## Generate Sequence

In [8]:
from numpy import array
from numpy import cumsum
from random import random
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

In [9]:
# create a sequence classification instance
def get_sequence(n_timesteps):
  # create a sequence of random numbers in [0,1]
  X = array([random() for _ in range(n_timesteps) ])
  # claculate the cutt-off value to change class values
  limit = n_timesteps/4.0
  # determine the class outcome for each item in coumulative sequence
  y = array([0 if x < limit else 1 for x in cumsum(X)])
  return X,y

In [14]:
def get_sequences(n_sequences, n_timesteps):
  seqX, seqY = list(), list()
  # create and store sequences 
  for _ in range(n_sequences):
    X, y = get_sequence(n_timesteps)
    seqX.append(X)
    seqY.append(y)
  seqX = array(seqX).reshape(n_sequences,n_timesteps,1)
  seqY = array(seqY).reshape(n_sequences,n_timesteps,1)
  return seqX, seqY

In [11]:
# test the get_sequence function
X, y = get_sequence(10)
print(X)
print(y)


[0.32170967 0.25877311 0.59533143 0.32714522 0.85045021 0.97755519
 0.73763427 0.95081814 0.07313794 0.58029161]
[0 0 0 0 0 1 1 1 1 1]


# Define and fit the model

In [12]:
# define problem
n_timesteps = 10
# define LSTM
model = Sequential()
model.add(Bidirectional(LSTM(50,return_sequences=True),input_shape=(n_timesteps,1)))
model.add(TimeDistributed(Dense(1,activation='sigmoid')))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 100)           20800     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             101       
Total params: 20,901
Trainable params: 20,901
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
# train LSTM
X, y = get_sequences(50000, n_timesteps)
model.fit(X, y, epochs=1, batch_size=10)

5000/5000 [==============================] - 33s 6ms/step - loss: 0.1062 - acc: 0.9602


In [17]:
# evaluate LSTM
X, y = get_sequences(100, n_timesteps)
loss, acc = model.evaluate(X, y, verbose=0)
print( 'Loss: %f, Accuracy: %f' % (loss, acc*100))


Loss: 0.025953, Accuracy: 99.100000


In [18]:
# make predictions
for _ in range(10):
  X, y = get_sequences(1, n_timesteps)
  yhat = model.predict_classes(X, verbose=0)
  exp, pred = y.reshape(n_timesteps), yhat.reshape(n_timesteps)
  print('y=%s, yhat=%s, correct=%s' % (exp, pred, array_equal(exp,pred)))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=False
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 0 1 1 1], yhat=[0 0 0 0 0 0 0 1 1 1], correct=True
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 0 0 1 1], yhat=[0 0 0 0 0 0 0 0 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=True
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 0 1 1 1], yhat=[0 0 0 0 0 0 0 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
